In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
df_kbli_real = pd.read_csv('../dataset/clean/clean_sak_occupation_kalteng.csv',dtype={"kbli_code": str, "kbli_label": str, "kbji_code": str, "kbji_label": str})

# Salin kolom yang dibutuhkan untuk klasifikasi KBLI
df_kbli = df_kbli_real[["text_description", "kbli_code"]].dropna()
# Tampilkan hasil
df_kbli.head(10)
# Pastikan kode dalam bentuk string agar bisa di-slice
df_kbli["kbli_code"] = df_kbli["kbli_code"].astype(str).str.strip()





df_kbli_code=df_kbli[["kbli_code"]]

df_kbli.head(10)

,text_description,kbli_code
0,Aktivitas: Karyawan panen kelapa sawit di kebu...,01262
1,"Aktivitas: Karyawan perawatan (memupuk, nyempr...",01262
2,Aktivitas: Ketua koperasi kelompok tani mitra ...,01262
3,"Aktivitas: Merawat, membersihkan, memupuk kebu...",01262
4,Aktivitas: Pemilik dan penjual hasil kebun kel...,01262
5,Aktivitas: Membantu suami mengurus kebun kepa ...,01262
6,Aktivitas: Pemilik dan penjual kebutuhan rumah...,47599
7,Aktivitas: Pemilik toko kelontong dan sembako....,47112
8,Aktivitas: Mandor karyawan perawatan kebun kel...,01262
9,"Aktivitas: Karyawan perawatan (Memupuk, meneba...",01262


In [ ]:
### load kbli labe description

kbli_label=pd.read_csv("../dataset/kbli2020.csv")

kbli_label.head()




,kategori,digit,kode,judul,deskripsi
0,A,1,A,"Pertanian, Kehutanan dan Perikanan",Kategori ini mencakup semua kegiatan ekonomi/l...
1,A,2,01,"Pertanian Tanaman, Peternakan, Perburuan dan K...",Golongan pokok ini mencakup pertanian tanaman ...
2,A,3,011,Pertanian Tanaman Semusim,Golongan ini mencakup penanaman tanaman yang t...
3,A,4,0111,"Pertanian serealia (bukan padi), aneka kacang ...",Subgolongan ini mencakup pertanian semua serea...
4,A,5,01111,Pertanian Jagung,Kelompok ini mencakup usaha pertanian komodita...


In [4]:
kbli_label[kbli_label['kode']=="01262"]



,kategori,digit,kode,judul,deskripsi
53,A,5,01262,Perkebunan Buah Kelapa Sawit,Kelompok ini mencakup usaha perkebunan mulai d...


In [5]:
def extract_hierarki_kbli(kbli_code):
    #code=kbli_code["kbli_code"]
    code = str(kbli_code["kbli_code"])
    #print(f"code:{code}")
    kbli_label_5_df=kbli_label[kbli_label['kode']==code]
    #print(f"kbli_kategori:{kbli_label_5_df['kategori'].values[0]}")
    kategori=kbli_label_5_df['kategori'].values[0]
    # print(kbli_label_5_df)
    kbli_code_5=code[:5]
    kbli_code_4=code[:4]
    kbli_code_3=code[:3]
    kbli_code_2=code[:2]
    return kategori,kbli_code_2, kbli_code_3, kbli_code_4, code




In [6]:
temp_hicode=df_kbli_code.apply(extract_hierarki_kbli,axis=1)
# Ubah ke dalam list kemudian ubah ke DataFrame
df_temp = pd.DataFrame(temp_hicode.tolist(), columns=['kategori','kbli_code_2', 'kbli_code_3', 'kbli_code_4', 'kbli_code_5'])
# # Gabungkan dengan dataframe awal df_kbli_code
df_hierarki_kbli = pd.DataFrame()  # misalnya sebelumnya sudah ada kolom lain
df_hierarki_kbli[['kategori','kbli_code_2', 'kbli_code_3', 'kbli_code_4', 'kbli_code_5']] = df_temp
# # Cek hasil
df_hierarki_kbli.tail()

,kategori,kbli_code_2,kbli_code_3,kbli_code_4,kbli_code_5
11737,A,03,031,0312,03125
11738,B,07,073,0730,07301
11739,G,47,479,4799,47999
11740,B,07,073,0730,07301
11741,A,02,023,0230,02301


In [103]:
print(kbli_label.columns)


Index(['kategori', 'digit', 'kode', 'judul', 'deskripsi'], dtype='object')


In [104]:
# Pastikan kolom kode di kbli_label bertipe string
kbli_label["kode"] = kbli_label["kode"].astype(str)
kbli_label["digit"] = kbli_label["digit"].astype(int)

# 1. kategori_desc
df_hierarki_kbli = df_hierarki_kbli.merge(
    kbli_label[kbli_label["digit"] == 1][["kode","judul","deskripsi"]],
    left_on="kategori", right_on="kode", how="left"
).rename(columns={"deskripsi": "kategori_desc","judul":"kategori_title"}).drop(columns="kode")

# 2. kbli_code_2_desc
df_hierarki_kbli = df_hierarki_kbli.merge(
    kbli_label[kbli_label["digit"] == 2][["kode","judul","deskripsi"]],
    left_on="kbli_code_2", right_on="kode", how="left"
).rename(columns={"deskripsi": "kbli_code_2_desc","judul":"kbli_code_2_title"}).drop(columns="kode")

# 3. kbli_code_3_desc
df_hierarki_kbli = df_hierarki_kbli.merge(
    kbli_label[kbli_label["digit"] == 3][["kode","judul","deskripsi"]],
    left_on="kbli_code_3", right_on="kode", how="left"
).rename(columns={"deskripsi": "kbli_code_3_desc","judul":"kbli_code_3_title"}).drop(columns="kode")

# 4. kbli_code_4_desc
df_hierarki_kbli = df_hierarki_kbli.merge(
    kbli_label[kbli_label["digit"] == 4][["kode","judul","deskripsi"]],
    left_on="kbli_code_4", right_on="kode", how="left"
).rename(columns={"deskripsi": "kbli_code_4_desc","judul":"kbli_code_4_title"}).drop(columns="kode")

# 5. kbli_code_5_desc
df_hierarki_kbli = df_hierarki_kbli.merge(
    kbli_label[kbli_label["digit"] == 5][["kode","judul","deskripsi"]],
    left_on="kbli_code_5", right_on="kode", how="left"
).rename(columns={"deskripsi": "kbli_code_5_desc","judul":"kbli_code_5_title"}).drop(columns="kode")


In [96]:
df_hierarki_kbli

,kategori,kbli_code_2,kbli_code_3,kbli_code_4,kbli_code_5,kategori_desc,kbli_code_2_desc,kbli_code_3_desc,kbli_code_4_desc,kbli_code_5_desc
0,A,01,012,0126,01262,Kategori ini mencakup semua kegiatan ekonomi/l...,Golongan pokok ini mencakup pertanian tanaman ...,Golongan ini mencakup penanaman tanaman yang b...,Subgolongan ini mencakup : \n- Perkebunan bua...,Kelompok ini mencakup usaha perkebunan mulai d...
1,A,01,012,0126,01262,Kategori ini mencakup semua kegiatan ekonomi/l...,Golongan pokok ini mencakup pertanian tanaman ...,Golongan ini mencakup penanaman tanaman yang b...,Subgolongan ini mencakup : \n- Perkebunan bua...,Kelompok ini mencakup usaha perkebunan mulai d...
2,A,01,012,0126,01262,Kategori ini mencakup semua kegiatan ekonomi/l...,Golongan pokok ini mencakup pertanian tanaman ...,Golongan ini mencakup penanaman tanaman yang b...,Subgolongan ini mencakup : \n- Perkebunan bua...,Kelompok ini mencakup usaha perkebunan mulai d...
3,A,01,012,0126,01262,Kategori ini mencakup semua kegiatan ekonomi/l...,Golongan pokok ini mencakup pertanian tanaman ...,Golongan ini mencakup penanaman tanaman yang b...,Subgolongan ini mencakup : \n- Perkebunan bua...,Kelompok ini mencakup usaha perkebunan mulai d...
4,A,01,012,0126,01262,Kategori ini mencakup semua kegiatan ekonomi/l...,Golongan pokok ini mencakup pertanian tanaman ...,Golongan ini mencakup penanaman tanaman yang b...,Subgolongan ini mencakup : \n- Perkebunan bua...,Kelompok ini mencakup usaha perkebunan mulai d...
...,...,...,...,...,...,...,...,...,...,...
11737,A,03,031,0312,03125,Kategori ini mencakup semua kegiatan ekonomi/l...,Golongan pokok ini mencakup penangkapan dan bu...,"Golongan ini mencakup kegiatan ""penangkapan ik...",Subgolongan ini mencakup: \n- Penangkapan ika...,Kelompok ini mencakup usaha atau kegiatan pena...
11738,B,07,073,0730,07301,Kategori ini mencakup kegiatan ekonomi/lapanga...,Golongan pokok ini mencakup pertambangan bijih...,Golongan ini mencakup pertambangan bijih logam...,Subgolongan ini mencakup pertambangan bijih lo...,"Kelompok ini mencakup usaha pertambangan, pemb..."
11739,G,47,479,4799,47999,Kategori ini meliputi kegiatan ekonomi/lapanga...,Barang yang dijual dalam golongan pokok ini di...,Golongan ini mencakup kegiatan perdagangan ece...,Subgolongan ini mencakup :\n- Perdagangan ecer...,Kelompok ini mencakup usaha perdagangan eceran...
11740,B,07,073,0730,07301,Kategori ini mencakup kegiatan ekonomi/lapanga...,Golongan pokok ini mencakup pertambangan bijih...,Golongan ini mencakup pertambangan bijih logam...,Subgolongan ini mencakup pertambangan bijih lo...,"Kelompok ini mencakup usaha pertambangan, pemb..."


In [ ]:
def extract_hierarki_kbli(kbli_code):
    #code=kbli_code["kbli_code"]
    code = str(kbli_code["kbli_code"])
    #print(f"code:{code}")
    kbli_label_5_df=kbli_label[kbli_label['kode']==code]
    #print(f"kbli_kategori:{kbli_label_5_df['kategori'].values[0]}")
    kategori=kbli_label_5_df['kategori'].values[0]
    # print(kbli_label_5_df)
    kbli_code_5=code[:5]
    kbli_code_4=code[:4]
    kbli_code_3=code[:3]
    kbli_code_2=code[:2]
    return kategori,kbli_code_2, kbli_code_3, kbli_code_4, code

def join_label(df):
    pass
